## Interactive Visual Analytics with Folium

In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
import io
import requests
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
if response.status_code == 200:
    spacex_csv_file = io.BytesIO(response.content)
    spacex_df = pd.read_csv(spacex_csv_file)
    print(spacex_df.head(5))
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [7]:
for index, row in launch_sites_df.iterrows():
    # Create a Circle for the launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#d35400',
        fill=True,
        fill_color='#d35400',
        fill_opacity=0.6,
        popup=row['Launch Site']
    ).add_to(site_map)
    
    # Create a Marker for the launch site
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    ).add_to(site_map)
site_map

In [8]:
for index, row in launch_sites_df.iterrows():
    # Create a Circle for the launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#d35400',
        fill=True,
        fill_color='#d35400',
        fill_opacity=0.6,
        popup=row['Launch Site']
    ).add_to(site_map)
    
    # Create a Marker for the launch site
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    ).add_to(site_map)

# Display the map in the notebook
site_map

In [9]:
site_map = folium.Map(location=[32.222, -90.999], zoom_start=4)
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)
for index, record in spacex_df.iterrows():
    marker_color = 'green' if record['class'] == 1 else 'red'
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=marker_color),
        popup=f"Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)
site_map

In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
from geopy.distance import geodesic

# Coordinates for KSC LC-39A launch site
launch_site_coords = (28.573255, -80.646895)

# Coordinates for a nearby coastline point
coastline_coords = (28.56367, -80.57163)

# Coordinates for a nearby highway point
highway_coords = (28.56367, -80.57163)

# Coordinates for a nearby railway point
railway_coords = (28.56230, -80.57736)

In [12]:
# Calculate the distances in kilometers
distance_coastline = geodesic(launch_site_coords, coastline_coords).km
distance_highway = geodesic(launch_site_coords, highway_coords).km
distance_railway = geodesic(launch_site_coords, railway_coords).km

print(f"Distance to Coastline: {distance_coastline:.2f} km")
print(f"Distance to Highway: {distance_highway:.2f} km")
print(f"Distance to Railway: {distance_railway:.2f} km")

Distance to Coastline: 7.44 km
Distance to Highway: 7.44 km
Distance to Railway: 6.91 km


In [13]:
from folium.features import DivIcon

# Assuming your 'site_map' is already created and centered

# Draw a line from the launch site to the coastline
folium.PolyLine(
    [launch_site_coords, coastline_coords],
    color='blue',
    weight=2.5,
    opacity=1
).add_to(site_map)

# Add a marker with the distance label for the coastline
folium.Marker(
    location=coastline_coords,
    icon=DivIcon(
        icon_size=(150, 30),
        icon_anchor=(-20, 20),
        html=f'<div style="font-size: 10pt; color:blue;"><b>{distance_coastline:.2f} km</b></div>'
    )
).add_to(site_map)

# Display the map
site_map

In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [16]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Define the provided distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Coordinates for CCAFS LC-40 launch site
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coordinates for the closest coastline point (using your provided example)
coastline_lat = 28.56367
coastline_lon = -80.56790

# Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"The distance to the closest coastline from CCAFS LC-40 is: {distance_coastline:.2f} km")

# Assuming your 'site_map' object is already initialized
# Let's create a temporary map for demonstration purposes if it doesn't exist
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a PolyLine object to draw a line between the launch site and the coastline
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='blue',
    weight=2.5,
    opacity=1
).add_to(site_map)

# Create a marker on the coastline to display the distance
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 30),
        icon_anchor=(0, 0),
        html='<div style="font-size: 10pt; color:blue;"><b>%s</b></div>' % f'{distance_coastline:.2f} km'
    )
).add_to(site_map)

# Display the map
site_map

The distance to the closest coastline from CCAFS LC-40 is: 0.87 km


In [17]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Define the provided distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Coordinates for CCAFS LC-40 launch site
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Coordinates for the closest coastline point (using your provided example)
coastline_lat = 28.56367
coastline_lon = -80.56790

# Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Assuming your 'site_map' object is already initialized
# Create a temporary map for demonstration if it doesn't exist
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a Marker on the coastline point to display the distance
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 30),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 10pt; color:blue;"><b>{distance_coastline:.2f} km</b></div>'
    )
)

# Add the marker to the map
distance_marker.add_to(site_map)

# Display the map
site_map

In [18]:
import folium

# Define the coordinates for the launch site and coastline
launch_site_coords = [28.563197, -80.576820]
coastline_coords = [28.56367, -80.56790]

# Assuming your 'site_map' object is already initialized
# Create a temporary map for demonstration if it doesn't exist
site_map = folium.Map(location=[launch_site_coords[0], launch_site_coords[1]], zoom_start=15)

# Create a PolyLine object
# The 'locations' parameter takes a list of coordinate pairs
lines = folium.PolyLine(
    locations=[launch_site_coords, coastline_coords],
    weight=2.5,
    color='blue',
)

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map

In [19]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Define the provided distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Coordinates for the CCAFS LC-40 launch site
launch_site_coords = [28.563197, -80.576820]

# Sample coordinates for nearby proximities
city_coords = [28.5383, -81.3792]  # Orlando, Florida
railway_coords = [28.57218, -80.58525]
highway_coords = [28.56385, -80.57014]

# Calculate the distances
distance_city = calculate_distance(launch_site_coords[0], launch_site_coords[1], city_coords[0], city_coords[1])
distance_railway = calculate_distance(launch_site_coords[0], launch_site_coords[1], railway_coords[0], railway_coords[1])
distance_highway = calculate_distance(launch_site_coords[0], launch_site_coords[1], highway_coords[0], highway_coords[1])

# Assuming 'site_map' is already initialized
# Create a temporary map for demonstration if it doesn't exist
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# Create and add a PolyLine and Marker for the City
folium.PolyLine(
    locations=[launch_site_coords, city_coords],
    color='purple',
    weight=2,
    opacity=1
).add_to(site_map)
folium.Marker(
    location=city_coords,
    icon=DivIcon(
        icon_size=(150, 30),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 10pt; color:purple;"><b>City: {distance_city:.2f} km</b></div>'
    )
).add_to(site_map)

# Create and add a PolyLine and Marker for the Railway
folium.PolyLine(
    locations=[launch_site_coords, railway_coords],
    color='green',
    weight=2,
    opacity=1
).add_to(site_map)
folium.Marker(
    location=railway_coords,
    icon=DivIcon(
        icon_size=(150, 30),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 10pt; color:green;"><b>Railway: {distance_railway:.2f} km</b></div>'
    )
).add_to(site_map)

# Create and add a PolyLine and Marker for the Highway
folium.PolyLine(
    locations=[launch_site_coords, highway_coords],
    color='orange',
    weight=2,
    opacity=1
).add_to(site_map)
folium.Marker(
    location=highway_coords,
    icon=DivIcon(
        icon_size=(150, 30),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 10pt; color:orange;"><b>Highway: {distance_highway:.2f} km</b></div>'
    )
).add_to(site_map)

# Display the map
site_map

In [20]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# --- Distance Calculation Function ---
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# --- Coordinates ---
# CCAFS LC-40 Launch Site
launch_site_coords = [28.563197, -80.576820]
# Closest Coastline Point (Short Distance - Safety Requirement)
coastline_coords = [28.56367, -80.56790]
# Closest Railway Point (Logistical Requirement)
railway_coords = [28.57218, -80.58525]
# Closest Highway Point (Logistical Requirement)
highway_coords = [28.56385, -80.57014]
# Closest City (Orlando, Florida - Safety Requirement)
city_coords = [28.5383, -81.3792]

# --- Calculate Distances ---
distance_coastline = calculate_distance(launch_site_coords[0], launch_site_coords[1], coastline_coords[0], coastline_coords[1])
distance_railway = calculate_distance(launch_site_coords[0], launch_site_coords[1], railway_coords[0], railway_coords[1])
distance_highway = calculate_distance(launch_site_coords[0], launch_site_coords[1], highway_coords[0], highway_coords[1])
distance_city = calculate_distance(launch_site_coords[0], launch_site_coords[1], city_coords[0], city_coords[1])

# --- Map Initialization ---
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# --- Function to add PolyLine and Marker ---
def add_proximity_to_map(map_obj, target_coords, proximity_coords, distance, label, color):
    # Draw PolyLine
    folium.PolyLine(
        locations=[target_coords, proximity_coords],
        color=color,
        weight=2,
        opacity=1
    ).add_to(map_obj)

    # Add Distance Marker
    folium.Marker(
        location=proximity_coords,
        icon=DivIcon(
            icon_size=(150, 30),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 10pt; color:{color};"><b>{label}: {distance:.2f} km</b></div>'
        )
    ).add_to(map_obj)

# --- Add All Proximities to Map ---

# Coastline (Blue - Shortest distance for safety)
add_proximity_to_map(site_map, launch_site_coords, coastline_coords, distance_coastline, "Coastline", "blue")

# Railway (Green - Essential for large component logistics)
add_proximity_to_map(site_map, launch_site_coords, railway_coords, distance_railway, "Railway", "green")

# Highway (Orange - Essential for general logistics)
add_proximity_to_map(site_map, launch_site_coords, highway_coords, distance_highway, "Highway", "orange")

# City (Purple - Longest distance for public safety)
add_proximity_to_map(site_map, launch_site_coords, city_coords, distance_city, "City (Orlando)", "purple")

# Display the map
site_map